# Test noise cross-correlations in spectra version 3.1.1

## First generate spectra using `so_noise_models`

In [ ]:
from so_models_v3 import SO_Noise_Calculator_Public_v3_1_1 as so_models
import os
import matplotlib
matplotlib.rc('text', usetex=True)
fontProperties = {
                  'weight' : 'normal', 'size' : 16}
import matplotlib.pyplot as plt

####################################################################
####################################################################
##                   demonstration of the code
####################################################################

mode=1 # baseline
fsky=1
ellmax=1e4
el=50.

dset_label = 'LAT\\_V3.1'
lat = so_models.SOLatV3point1(mode, el=el)
corr_pairs = [(0,1),(2,3),(4,5)]

print(dset_label)
bands = lat.get_bands()
print("band centers: ", lat.get_bands(), "[GHz]")
print("beam sizes: "  , lat.get_beams(), "[arcmin]")
N_bands = len(bands)

ell, N_ell_LA_T_full,N_ell_LA_P_full = lat.get_noise_curves(
    fsky, ellmax, 1, full_covar=True, deconv_beam=False)

WN_levels = lat.get_white_noise(fsky)**.5

N_ell_LA_T  = N_ell_LA_T_full[range(N_bands),range(N_bands)]
N_ell_LA_Tx = [N_ell_LA_T_full[i,j] for i,j in corr_pairs]
N_ell_LA_P  = N_ell_LA_P_full[range(N_bands),range(N_bands)]
N_ell_LA_Px = [N_ell_LA_P_full[i,j] for i,j in corr_pairs]

print("white noise levels: "  , WN_levels, "[uK-arcmin]")

target = str(lat.__class__.__name__).split('.')[-1]

In [ ]:
%matplotlib inline

In [ ]:
import healpy as hp

In [ ]:
nside = 256
lmax = 3 * nside -1

## Simulate maps with `mapsims`

In [ ]:
from mapsims import SONoiseSimulator
sim = SONoiseSimulator(nside=256,
                       rolloff_ell=50, full_covariance=True, homogeneous=True,
                       apply_beam_correction=False)

In [ ]:
tube = "LT6"

In [ ]:
m = sim.simulate(tube=tube, seed=888)

In [ ]:
# Check if we are on Travis-CI
on_CI = os.environ.get("CI", "false") == "true"

In [ ]:
if not on_CI:
    hp.mollview(m[0][0][0], min=-100, max=100)

## Take spectra and cross-spectra of the maps and compare them

In [ ]:
cross_noise_cl = hp.anafast(m[0][0], m[1][0],
                            lmax=lmax, use_pixel_weights=True)

In [ ]:
cross_noise_cl.shape

In [ ]:
corr_pairs

In [ ]:
bands

In [ ]:
noise_cl_27 = hp.anafast(m[0][0],
                            lmax=lmax, use_pixel_weights=True)

In [ ]:
if not on_CI:
    plt.figure()
    for i in range(2):
        plt.loglog(ell,N_ell_LA_T[i], label='%i GHz (%s)' % (bands[i], dset_label),
                   ls='-', lw=2.)

    # include correlated atmospheric noise across frequencies
    for _c,(i,j) in enumerate(corr_pairs[:1]):
        plt.loglog(ell, N_ell_LA_T_full[i,j],
                   label=r'$%i \times %i$ GHz atm.' % (bands[i],bands[j]),
                   lw=1.5)

    plt.loglog(cross_noise_cl[0], lw=1, label="Cross spectra from maps")
    plt.loglog(noise_cl_27[0], lw=1, label="27 GHz spectrum from maps")

    plt.title(r"$N(\ell$) Temperature", fontsize=18)
    plt.ylabel(r"$N(\ell$) [$\mu$K${}^2$]", fontsize=16)
    plt.xlabel(r"$\ell$", fontsize=16)
    plt.ylim(5e-7,1e2)
    plt.xlim(0,10000)
    plt.legend(loc='lower left', ncol=2, fontsize=8)
    plt.grid()

In [ ]:
ell

In [ ]:
import numpy as np

In [ ]:
np.testing.assert_allclose(cross_noise_cl[0][80:200],
                           N_ell_LA_T_full[0,1][80+2:200+2],
                           rtol=.5)

In [ ]:
np.testing.assert_allclose(noise_cl_27[0][80:200],
                           N_ell_LA_T_full[0,0][80+2:200+2],
                           rtol=.6)

## Polarization

In [ ]:
if not on_CI:

    plt.figure(figsize=(10,5))
    #for i in range(2):
    #    plt.loglog(ell,N_ell_LA_P[i], label='%i GHz (%s)' % (bands[i], dset_label),
    #               color=colors[i], ls='-', lw=2.)

    # include correlated atmospheric noise across frequencies
    for _c,(i,j) in enumerate(corr_pairs[:1]):
        plt.loglog(ell, N_ell_LA_P_full[i,j],
                   label=r'$%i \times %i$ GHz atm.' % (bands[i],bands[j]),
                   lw=2)

    plt.loglog(cross_noise_cl[1], lw=1, label="EE Cross spectrum from maps")
    plt.loglog(cross_noise_cl[2], lw=1, label="BB Cross spectrum from maps")

    plt.title(r"$N(\ell$) Polarization", fontsize=18)
    plt.ylabel(r"$N(\ell$) [$\mu$K${}^2$]", fontsize=16)
    plt.xlabel(r"$\ell$", fontsize=16)
    plt.ylim(1e-5, 100)
    plt.xlim(0,1000)
    plt.legend(loc='lower left', ncol=2, fontsize=8)
    plt.grid()

In [ ]:
np.testing.assert_allclose(noise_cl_27[1][80:300],
                           N_ell_LA_P_full[0,0][80+2:300+2],
                           rtol=.5)

In [ ]:
np.testing.assert_allclose(noise_cl_27[2][80:300],
                           N_ell_LA_P_full[0,0][80+2:300+2],
                           rtol=.5)

In [ ]:
np.testing.assert_allclose(cross_noise_cl[1][80:300],
                           N_ell_LA_P_full[0,1][80+2:300+2],
                           rtol=.5)

In [ ]:
np.testing.assert_allclose(cross_noise_cl[2][80:300],
                           N_ell_LA_P_full[0,1][80+2:300+2],
                           rtol=.5)

## Test that the seed is working

result should be deterministic

In [ ]:
expected_cross_noise_cl_2 = np.array([0.01713746, 0.01950811, 0.01701195, 0.01826676, 0.01783291,
       0.01762982, 0.0248187 , 0.01548188, 0.01839568, 0.01786226])

In [ ]:
np.testing.assert_allclose(expected_cross_noise_cl_2, cross_noise_cl[2][80:90], rtol=1e-6)

## EB XCorr should be consistent with zero

In [ ]:
if not on_CI:
    plt.figure(figsize=(10,5))
    plt.semilogx(cross_noise_cl[5], lw=1, label="EB Cross spectrum from maps")

    plt.title(r"$N(\ell$) Polarization", fontsize=18)
    plt.ylabel(r"$N(\ell$) [$\mu$K${}^2$]", fontsize=16)
    plt.xlabel(r"$\ell$", fontsize=16)
    plt.legend(loc='lower left', ncol=2, fontsize=8)
    plt.grid()

In [ ]:
assert np.std(cross_noise_cl[5]) < 1e-2